In [1]:
import pandas as pd
import py_entitymatching as em
A = em.read_csv_metadata('./TableA_lower.csv', key = 'ID')
B = em.read_csv_metadata('./TableB_lower.csv', key = 'ID')

block_f = em.get_features_for_blocking(A, B)
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
r = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', block_t, block_s)
em.add_feature(block_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', r)

ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'Author', 'Author', 
                    l_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    r_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    overlap_size = 2)

D = ob.block_candset(C, 'Title', 'Title', overlap_size = 4)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Author,Author,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
3,Publication,Publication,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
4,Format,Format,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
5,ISBN,ISBN,numeric,numeric,Exact Match; Absolute Norm
6,Series,Series,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Physical Details,Physical Details,short string (1 word),short string (1 word to 5 words),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
A.shape

In [2]:
label_S = pd.read_csv('./Set_G.csv')
# em.copy_properties(S, label_S)
em.set_property(label_S, 'key', '_id')
em.set_property(label_S, 'fk_ltable', 'ltable_ID')
em.set_property(label_S, 'fk_rtable', 'rtable_ID')
label_S_rtable = em.read_csv_metadata('./label_S_rtable.csv')
label_S_ltable = em.read_csv_metadata('./label_S_ltable.csv')
em.set_property(label_S, 'rtable', label_S_rtable)
em.set_property(label_S, 'ltable', label_S_ltable)
label_S['ltable_Title'] = label_S['ltable_Title'].apply(lambda x: x.lower())
label_S['rtable_Title'] = label_S['rtable_Title'].apply(lambda x: x.lower())
label_S['ltable_Author'] = label_S['ltable_Author'].apply(lambda x: x.lower())
label_S['rtable_Author'] = label_S['rtable_Author'].apply(lambda x: x.lower())
label_S['ltable_Publication'] = label_S['ltable_Publication'].apply(lambda x: x.lower())
label_S['rtable_Publication'] = label_S['rtable_Publication'].apply(lambda x: x.lower())
label_S['ltable_Format'] = label_S['ltable_Format'].apply(lambda x: x.lower())
label_S['rtable_Format'] = label_S['rtable_Format'].apply(lambda x: x.lower())
label_S['ltable_Series'] = label_S['ltable_Series'].apply(lambda x: x.lower())
label_S['rtable_Series'] = label_S['rtable_Series'].apply(lambda x: x.lower())

In [5]:
label_S

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Author,ltable_Publication,ltable_Format,ltable_ISBN,ltable_Series,ltable_Physical Details,rtable_Title,rtable_Author,rtable_Publication,rtable_Format,rtable_ISBN,rtable_Series,rtable_Physical Details,label
0,2,a5712,b297,"""intelligence science and big data engineering : 7th international conference, iscide 2017, dali...","""yi sun, huchuan lu, lihe zhang, jian yang, hua huang (eds.)""","""cham, switzerland : springer, 2017.""","""books""",9783319677774,"""lecture notes in computer science ; 10559,lncs sublibrary. image processing, computer vision, p...","""nan""","""intelligent science and intelligent data engineering : third sino-foreign-interchange workshop,...","""yang, jian, | fang, fang. | sun, changyin.""","""berlin ; springer, â©2013.""","""electronic books. | conference papers and proceedings.""",9783642366697,"""lecture notes in computer science ; 7751. lncs sublibrary. sl 6, image processing, computer vis...","""1 online resource.""",0
1,3,a3796,b297,"""cloud computing and security : first international conference, icccs 2015, nanjing, china, augu...","""zhiqiu huang, xingming sun, junzhou luo, jian wang (eds.)""","""cham : springer, 2015.""","""books""",9783319270517,"""lecture notes in computer science ; 9483,lncs sublibrary. information systems and applications,...","""nan""","""intelligent science and intelligent data engineering : third sino-foreign-interchange workshop,...","""yang, jian, | fang, fang. | sun, changyin.""","""berlin ; springer, â©2013.""","""electronic books. | conference papers and proceedings.""",9783642366697,"""lecture notes in computer science ; 7751. lncs sublibrary. sl 6, image processing, computer vis...","""1 online resource.""",0
2,5,a4637,b297,"""chinese computational linguistics and natural language processing based on naturally annotated ...","""edited by maosong sun, xuanjing huang, hongfei lin, zhiyuan liu, yang liu""","""cham, switzerland : springer, 2016.""","""books""",9783319476742,"""lecture notes in computer science. lecture notes in artificial intelligence ; 10035,lncs sublib...","""nan""","""intelligent science and intelligent data engineering : third sino-foreign-interchange workshop,...","""yang, jian, | fang, fang. | sun, changyin.""","""berlin ; springer, â©2013.""","""electronic books. | conference papers and proceedings.""",9783642366697,"""lecture notes in computer science ; 7751. lncs sublibrary. sl 6, image processing, computer vis...","""1 online resource.""",0
3,6,a1971,b4754,"""advances in visual computing : first international symposium, isvc 2005, lake tahoe, nv, usa, d...","""george bebis ... [and others] (eds.)""","""berlin ; new york : springer, 2005.""","""books""",9783540307501,"""lecture notes in computer science. 3804,""","""nan""","""advances in visual computing : 4th international symposium, isvc 2008, las vegas, nv, usa, dece...","""bebis, george.""","""berlin ; springer, c2008.""","""electronic books.""",9783540896463,"""lecture notes in computer science, 5358 0302-9743 ;""","""2 v.""",0
4,8,a3834,b4003,"""graph-grammars and their application to computer science : 2nd international workshop, haus ohr...","""under the auspices of the european association for theoretical computer science ; edited by har...","""berlin ; new york : springer-verlag, 1983.""","""books""",9780387123103,"""lecture notes in computer science ; 153,""","""nan""","""graph-grammars and their application to computer science and biology : international workshop, ...","""claus, volker, | ehrig, hartmut. | rozenberg, grzegorz.""","""berlin ; springer-verlag, 1979.""","""electronic books. | conference proceedings.""",9783540350910,"""lecture notes in computer science ; 73. 0302-9743""","""1 online resource (vii, 477 p.)""",0
5,13,a5626,b6767,"""computational intelligence in data mining. volume 2 : proceedings of the international conferen...","""himansu sekhar behera, durga prasad mohapatra, editors""","""india : springer, 2016.""","""bo

In [7]:
label_S['ltable_Title']

0      "intelligence science and big data engineering : 7th international conference, iscide 2017, dali...
1      "cloud computing and security : first international conference, icccs 2015, nanjing, china, augu...
2      "chinese computational linguistics and natural language processing based on naturally annotated ...
3      "advances in visual computing : first international symposium, isvc 2005, lake tahoe, nv, usa, d...
4      "graph-grammars and their application to computer science : 2nd international workshop, haus ohr...
5      "computational intelligence in data mining. volume 2 : proceedings of the international conferen...
6      "computational intelligence in data mining. volume 1 : proceedings of the international conferen...
7      "information security and privacy : second australasian conference, acisp '97, sydney, nsw, aust...
8                                                                 "graph data model and its data language"
9      "applied cryptography and netw

In [82]:
import re
# for Roman numerals matching
def Title_Title_blackbox_1(x, y):
    
#     regex_roman = '\s+[MDCLXVI]+\s+'
    regex_roman = '\s+[mdclxvi]+($|\s+)'
    x_match = None
    y_match = None
    if re.search(regex_roman, x):
        x_match = re.search(regex_roman, x).group(0).strip()
    if re.search(regex_roman, y):
        y_match = re.search(regex_roman, y).group(0).strip()

    print(x_match)
    print(y_match)
    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

str1 = "transactions on large-scale data- and knowledge centered systems vii "
str2 = "transactions on large-scale data- and knowledge-centered systems xxii"

Title_Title_blackbox_1(str1, str2)

vii
xxii


False

In [8]:
match_f = em.get_features_for_matching(A, B)
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
f1 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', match_t, match_s)
# f2 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Author"]), dlm_dc0(rtuple["Author"]))', match_t, match_s)
f3 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Publication"]), dlm_dc0(rtuple["Publication"]))', match_t, match_s)
f4 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Series"]), dlm_dc0(rtuple["Series"]))', match_t, match_s)
em.add_feature(match_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', f1)
# em.add_feature(match_f, 'Author_Author_jac_dlm_dc0_dlm_dc0', f2)
em.add_feature(match_f, 'Publication_Publication_jac_dlm_dc0_dlm_dc0', f3)
em.add_feature(match_f, 'Series_Series_jac_dlm_dc0_dlm_dc0', f4)


# Add blackbox feature

import re
# for Roman numerals matching
def Title_Title_blackbox_1(x, y):
    
    # get name attribute
    x_title = x['Title']
    y_title = y['Title']
#     regex_roman = '\s+[MDCLXVI]+\s+'
    regex_roman = '\s+[mdclxvi]+($|\s+)'
    x_match = None
    y_match = None
    if re.search(regex_roman, x_title):
        x_match = re.search(regex_roman, x_title).group(0).strip()
    if re.search(regex_roman, y_title):
        y_match = re.search(regex_roman, y_title).group(0).strip()

    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

em.add_blackbox_feature(match_f, 'blackbox_1', Title_Title_blackbox_1)


# for number matching (e.g. 6th edition)
def Title_Title_blackbox_2(x, y):
    # x, y will be of type pandas series
    
    x_title = x['Title']
    y_title = y['Title']
    regex_number = '\s+(\d+)\s*th'
    x_match = None
    y_match = None
    if re.search(regex_number, x_title):
        x_match = re.search(regex_number, x_title).group(1)
    if re.search(regex_number, y_title):
        y_match = re.search(regex_number, y_title).group(1)

    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

em.add_blackbox_feature(match_f, 'blackbox_2', Title_Title_blackbox_2)

# for number matching (e.g. 6th edition)
from fuzzywuzzy import fuzz
def Author_Author_blackbox_3(x, y):
    # x, y will be of type pandas series
    
    x_author = x['Author']
    y_author = y['Author']
    return fuzz.token_set_ratio(x_author, y_author)/100.0
    
em.add_blackbox_feature(match_f, 'blackbox_3', Author_Author_blackbox_3)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Author,Author,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
3,Publication,Publication,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
4,Format,Format,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
5,ISBN,ISBN,numeric,numeric,Exact Match; Absolute Norm
6,Series,Series,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Physical Details,Physical Details,short string (1 word),short string (1 word to 5 words),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


True

In [9]:
match_f = match_f[(match_f['left_attribute'] != 'ID') & (match_f['left_attribute'] != 'ISBN')]
match_f = match_f[(match_f['left_attribute'] != 'Format') & (match_f['left_attribute'] != 'Series')]

In [10]:
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
6,Title_Title_jac_qgm_3_qgm_3,Title,Title,qgm_3,qgm_3,jaccard,<function Title_Title_jac_qgm_3_qgm_3 at 0x7f0d01f7d488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Title_Title_cos_dlm_dc0_dlm_dc0,Title,Title,dlm_dc0,dlm_dc0,cosine,<function Title_Title_cos_dlm_dc0_dlm_dc0 at 0x7f0d01f7d400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
25,Title_Title_jac_dlm_dc0_dlm_dc0,Title,Title,dlm_dc0,dlm_dc0,jaccard,<function fn at 0x7f0d0205e158>,"def fn(ltuple, rtuple):\n return jaccard(dlm_dc0(ltuple[""Title""]), dlm_dc0(rtuple[""Title""]))",False
26,Publication_Publication_jac_dlm_dc0_dlm_dc0,Publication,Publication,dlm_dc0,dlm_dc0,jaccard,<function fn at 0x7f0d022e4d90>,"def fn(ltuple, rtuple):\n return jaccard(dlm_dc0(ltuple[""Publication""]), dlm_dc0(rtuple[""Publ...",False
28,blackbox_1,None,None,None,None,None,<function Title_Title_blackbox_1 at 0x7f0d022e4bf8>,None,False
29,blackbox_2,None,None,None,None,None,<function Title_Title_blackbox_2 at 0x7f0d022e4a60>,None,False
30,blackbox_3,None,None,None,None,None,<function Author_Author_blackbox_3 at 0x7f0d022e4950>,None,False


In [11]:
H = em.extract_feature_vecs(label_S, feature_table=match_f, attrs_after=['label'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [12]:
# RF
rf = em.RFMatcher(n_estimators = 300,max_depth = 300, name='RF')
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')



In [13]:
H_test = em.extract_feature_vecs(D, feature_table=match_f)
pred_table = rf.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'], 
                        target_attr='predicted_labels', 
                        return_probs=True, 
                        probs_attr='proba', 
                        append=True)
# eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
# eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [14]:
pred_rows = pred_table[pred_table['predicted_labels'] == 1]

In [15]:
pred_rows.shape

(1337, 12)

In [ ]:
pred_rows

In [20]:
matched_set = D[D['_id'].isin(pred_rows['_id'])]

In [21]:
matched_set[matched_set['ltable_ISBN'] == matched_set['rtable_ISBN']].shape

(654, 17)

In [24]:
matched_set
# matched_set['ltable_Title']

,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Author,ltable_Publication,ltable_Format,ltable_ISBN,ltable_Series,ltable_Physical Details,rtable_Title,rtable_Author,rtable_Publication,rtable_Format,rtable_ISBN,rtable_Series,rtable_Physical Details
0,0,a2337,b0,"""statistical learning and data science""","""edited by mireille gettler summa ... [and others]""","""boca raton : crc press, [2012] ©2012""","""books""",9781439867631,"""series in computer science and data analysis,""","""nan""","""statistical learning and data science""","""summa, mireille gettler.""","""boca raton""","""nan""",9781439867631,"""series in computer science and data analysis.""","""xv, 227 p."""
1,1,a3097,b1,"""intelligent techniques for data science""","""rajendra akerkar, priti srinivas sajja""","""cham, switzerland : springer, 2016.""","""books""",9783319292069,"""nan""","""nan""","""intelligent techniques for data science""","""akerkar, rajendra""","""cham, switzerland""","""electronic books.""",9783319292069,"""nan""","""1 online resource (xvi, 272 pages)"""
2,2,a4508,b2,"""algorithms for data science""","""brian steele, john chandler, swarna reddy""","""cham, switzerland : springer, 2016.""","""books""",9783319457970,"""nan""","""nan""","""algorithms for data science""","""steele, brian""","""cham, switzerland""","""electronic books.""",9783319457970,"""nan""","""1 online resource (xxiii, 430 pages)"""
3,3,a3267,b3,"""data science at the command line""","""jeroen janssens""","""first edition. sebastopol, ca : o'reilly, 2014. ©2015""","""books""",9781491947852,"""nan""","""nan""","""data science at the command line""","""janssens, jeroen""","""sebastopol, ca""","""nan""",9781491947852,"""nan""","""xvii, 191 pages"""
5,5,a4755,b5,"""introduction to hpc with mpi for data science""","""frank nielsen""","""cham : springer, 2016.""","""books""",9783319219035,"""undergraduate topics in computer science,""","""nan""","""introduction to hpc with mpi for data science""","""nielsen, frank""","""cham""","""electronic books.""",9783319219035,"""undergraduate topics in computer science, 1863-7310""","""1 online resource (xxxiii, 282 pages)"""
6,6,a2523,b6,"""data science using oracle data miner and oracle r enterprise : transform your business systems ...","""sibanjan das""","""berkeley, ca : apress, 2016. berkeley, ca : apress, 2016.""","""books""",9781484226148,"""nan""","""nan""","""data science using oracle data miner and oracle r enterprise : transform your business systems ...","""das, sibanjan.""","""berkeley, ca""","""electronic books.""",9781484226148,"""nan""","""1 online resource (300 pages)"""
7,7,a3584,b7,"""the data science handbook""","""field cady""","""hoboken, nj : john wiley & sons, inc., 2017.""","""books""",9781119092933,"""nan""","""nan""","""the data science handbook""","""cady, field, 1984-""","""hoboken, nj""","""electronic books. | handbooks and manuals.""",9781119092933,"""nan""","""1 online resource"""
8,8,a1869,b8,"""data science : create teams that ask the right questions and deliver real value""","""doug rose""","""[berkeley, ca] : apress, 2016.""","""books""",9781484222539,"""nan""","""nan""","""data science : create teams that ask the right questions and deliver real value""","""rose, doug, (agile coach)""","""[berkeley, ca]""","""electronic books.""",9781484222539,"""nan""","""1 online resource"""
9,9,a1186,b9,"""spatial big data science classification techniques for earth observation imagery""","""zhe jiang, shashi shekhar""","""cham : springer, 2017.""","""books""",9783319601953,"""nan""","""nan""","""spatial big data science : classification techniques for earth observation imagery""","""jiang, zhe.""","""cham""","""electronic books.""",9783319601953,"""nan""","""1 online resource"""
10,10,a3432,b11,"""practical data science cookbook : 89 hands-on recipes to help you complete real-world data scie...","""tony ojeda, sean patrick murphy, benjamin bengfort, abhijit dasgupta""","""birmingham, uk : packt publishing ltd., september 20

In [26]:
matched_id = matched_set[['ltable_ID', 'rtable_ID']]

In [72]:
# for index, row in matched_set.iterrows():
#     print row['c1'], row['c2']

In [96]:
ltitle = '"transactions on large-scale data- and knowledge centered systems vii"'.strip().replace(':',' ').replace('-', ' ')
rtitle = '"transactions on large-scale data- and knowledge-centered systems xxii"'.strip().replace(':',' ').replace('-', ' ')
ltitle = re.split(' +' ,ltitle)
rtitle = re.split(' +' ,rtitle)
em.jaccard(ltitle, rtitle)

0.8181818181818182

In [97]:
ltitle

['"transactions',
 'on',
 'large',
 'scale',
 'data',
 'and',
 'knowledge',
 'centered',
 'systems',
 'vii"']

In [98]:
rtitle

['"transactions',
 'on',
 'large',
 'scale',
 'data',
 'and',
 'knowledge',
 'centered',
 'systems',
 'xxii"']

In [106]:
def jaccard_title(x):
    ltitle = x['ltable_Title'].lower().strip().split(" ")
    rtitle = x['rtable_Title'].lower().strip().split(" ")
#     print(ltitle)
#     print(rtitle)
#     print('\n')
    return em.jaccard(ltitle, rtitle)
# def udf__(x):
#     print(x)
matched_set['jaccard'] = matched_set.apply(jaccard_title, axis = 1)

/home/xiuyuan/anaconda3/envs/fonduer/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [122]:
import re
def include_roman(x):
    
    # get name attribute
    x_title = x['ltable_Title']
    y_title = x['rtable_Title']
    
#     regex_roman = '\s+[MDCLXVI]+\s+'
    regex_roman = '\s+[mdclxvi]+($|\s+)'
    x_match = None
    y_match = None
    if re.search(regex_roman, x_title):
        x_match = re.search(regex_roman, x_title).group(0).strip()
    if re.search(regex_roman, y_title):
        y_match = re.search(regex_roman, y_title).group(0).strip()

    if x_match is None or y_match is None:
        return 0
    elif x_match == y_match:
        return 1
    else:
        return -1
    
matched_set['roman'] = matched_set.apply(include_roman, axis = 1)

/home/xiuyuan/anaconda3/envs/fonduer/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
matched_set[matched_set['roman'] == -1]

In [107]:
pd.set_option('display.max_rows', 1000)

In [ ]:
matched_set[['ltable_Title', 'rtable_Title', 'jaccard']]

In [ ]:
matched_set[(matched_set['jaccard'] < 0.55)]

In [ ]:
matched_set[(matched_set['jaccard'] > 0.55) & (matched_set['jaccard'] < 0.6)]

In [ ]:
D[D['_id'] == 623][['ltable_Author', 'rtable_Author']]

In [ ]:
matched_set.to_csv("matched_set.csv", )

In [ ]:
matched_set = merge_set.iloc[:,:17]

In [ ]:
matched_set

In [ ]:
matched_set.to_csv('matched_set.csv', sep=',')

In [ ]:
D.shape

In [ ]:
pred_rows.shape

In [ ]:
matched_set['ltable_ID_x']

In [ ]:
A[~A['ID'].isin(matched_set['ltable_ID_x'])]

In [ ]:
B[~B['ID'].isin(matched_set['rtable_ID_x'])]

In [ ]:
A['ID']

In [37]:
# AB_list=[['ltable_ID','rtable_ID'] for x in pred_table if predicted_labels==1]
AB_dict =  {item[0]: item[1] for item in AB_list}
BA_dict =  {item[1]: item[0] for item in AB_list}

In [35]:
AB_list = matched_id.values.tolist()

In [71]:
matched_id.shape

(1337, 2)

In [72]:
AB_dict

{'a1001': 'b1146',
 'a1019': 'b6701',
 'a104': 'b127',
 'a105': 'b2853',
 'a1051': 'b5608',
 'a1063': 'b548',
 'a107': 'b28',
 'a1075': 'b2194',
 'a1086': 'b105',
 'a1088': 'b4231',
 'a1097': 'b5904',
 'a1106': 'b6544',
 'a1107': 'b3946',
 'a1117': 'b2270',
 'a1120': 'b6846',
 'a1124': 'b5704',
 'a1136': 'b3622',
 'a1150': 'b198',
 'a1152': 'b636',
 'a1161': 'b381',
 'a1162': 'b2270',
 'a1164': 'b872',
 'a1167': 'b5610',
 'a1175': 'b1058',
 'a1176': 'b727',
 'a1180': 'b2166',
 'a1181': 'b1016',
 'a1186': 'b9',
 'a1187': 'b5347',
 'a1190': 'b3049',
 'a1194': 'b2222',
 'a120': 'b4255',
 'a1202': 'b882',
 'a1212': 'b689',
 'a1215': 'b3810',
 'a1216': 'b6237',
 'a1218': 'b6115',
 'a1227': 'b878',
 'a123': 'b730',
 'a1232': 'b912',
 'a1259': 'b523',
 'a1275': 'b67',
 'a1276': 'b539',
 'a1288': 'b190',
 'a1289': 'b6838',
 'a1298': 'b1873',
 'a13': 'b94',
 'a1301': 'b502',
 'a1304': 'b3743',
 'a1310': 'b940',
 'a1314': 'b576',
 'a1322': 'b153',
 'a1337': 'b1058',
 'a1344': 'b5428',
 'a1351': 

In [75]:
import csv

attr=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details']
with open("table_E.csv", 'w') as csvOut:
    writer = csv.writer(csvOut,delimiter = ',')
    for index, r in A.iterrows():
        row=[]
        if r['ID'] in AB_dict.keys():
            for j in attr:
                if len(str(r[j]))<len(str(B.loc[B['ID'] == AB_dict[r['ID']]][j])):
                    row.append(B.loc[B['ID'] == AB_dict[r['ID']]][j])
                else:
                    row.append(r[j])
            else:
                for j in attr:
                    row.append(r[j])
        print(row)
#         writer.writerow(row)
        
    for index, r in B.iterrows():
        row=[]
        if r['ID'] not in BA_dict.keys():
            for j in attr:
                row.append(r[j])
#         writer.writerow(row)

[]
[]
[]
[]
[]
[]
[4057    "indexing it all : the subject in the age of documentation, information, and data"
Name: Title, dtype: object, 4057    "day, ronald e., 1959-"
Name: Author, dtype: object, '"cambridge, massachusetts ; london, england : the mit press, 2014. ©2014"', 4057    "nan"
Name: Format, dtype: object, 4057    9780262028219
Name: ISBN, dtype: int64, 4057    "history and foundations of information science."
Name: Series, dtype: object, 4057    "xiv, 170 pages ; 24 cm."
Name: Physical Details, dtype: object, '"indexing it all : the subject in the age of documentation, information, and data"', '"ronald e. day"', '"cambridge, massachusetts ; london, england : the mit press, 2014. ©2014"', '"books"', 9780262322775, '"history and foundations of information science,"', '"nan"']
[]
[3866    "data clustering : algorithms and applications."
Name: Title, dtype: object, 3866    "aggarwal, charu c."
Name: Author, dtype: object, 3866    "hoboken"
Name: Publication, dtype: object, 3866

[2145    "transactions on computational science v : special issue on cognitive knowledge representation"
Name: Title, dtype: object, 2145    "gavrilova, marina l."
Name: Author, dtype: object, 2145    "berlin"
Name: Publication, dtype: object, 2145    "electronic books."
Name: Format, dtype: object, 2145    9783642020971
Name: ISBN, dtype: int64, 2145    "lecture notes in computer science ; 5540."
Name: Series, dtype: object, 2145    "1 online resource (xii, 236 p.)"
Name: Physical Details, dtype: object, '"transactions on computational science v : special issue on cognitive knowledge representation"', '"marina l. gavrilova ... [and others] (eds.)"', '"berlin ; new york : springer, [2009] ©2009"', '"books"', 9783642020971, '"lecture notes in computer science ; 5540,"', '"nan"']
[26    "the data science design manual"
Name: Title, dtype: object, 26    "skiena, steven s."
Name: Author, dtype: object, 26    "cham, switzerland"
Name: Publication, dtype: object, 26    "electronic books."
Na

[2783    "transactions on computational science iii"
Name: Title, dtype: object, 2783    "gavrilova, marina l. | tan, c. j. kenneth"
Name: Author, dtype: object, 2783    "heidelberg"
Name: Publication, dtype: object, 2783    "electronic books."
Name: Format, dtype: object, 2783    9783642002120
Name: ISBN, dtype: int64, 2783    "lecture notes in computer science ; 5300."
Name: Series, dtype: object, 2783    "1 online resource."
Name: Physical Details, dtype: object, '"transactions on computational science xiii"', '"marina l. gavrilova, c.j. kenneth tan (eds.)"', '"berlin ; new york : springer, [2011] ©2011"', '"books"', 9783642226199, '"lecture notes in computer science ; 6750,"', '"nan"']
[]
[]
[205    "life science data mining"
Name: Title, dtype: object, 205    "wong, stephen t. c. | li, chung-sheng,"
Name: Author, dtype: object, 205    "singapore ; world scientific, c2006."
Name: Publication, dtype: object, 205    "nan"
Name: Format, dtype: object, 205    981270065
Name: ISBN, dtyp

[2868    "integrated technologies for environmental monitoring and information production"
Name: Title, dtype: object, 2868    "harmancioglu, nilgun b."
Name: Author, dtype: object, '"dordrecht ; boston : kluwer academic publishers : published in cooperation with nato scientific affairs division, [2003] ©2003"', 2868    "electronic books."
Name: Format, dtype: object, 2868    9789401002318
Name: ISBN, dtype: int64, 2868    "nato science series. series iv, earth and environmental sciences ; 23."
Name: Series, dtype: object, 2868    "1 online resource (496 pages)."
Name: Physical Details, dtype: object, '"integrated technologies for environmental monitoring and information production"', '"edited by nilgun b. harmancioglu ... [and others]"', '"dordrecht ; boston : kluwer academic publishers : published in cooperation with nato scientific affairs division, [2003] ©2003"', '"books"', 9781402013980, '"nato science series. earth and environmental sciences ; v. 23,"', '"nan"']
[]
[]
[]
['"mana

[2161    "a developer's guide to data modeling for sql server : covering sql server 2005 and 2008"
Name: Title, dtype: object, 2161    "johnson, eric, 1978-"
Name: Author, dtype: object, 2161    "upper saddle river, n.j."
Name: Publication, dtype: object, 2161    "electronic books."
Name: Format, dtype: object, 2161    9780321497642
Name: ISBN, dtype: int64, 2161    "addison-wesley microsoft technology series."
Name: Series, dtype: object, 2161    "1 online resource (xix, 277 p.)"
Name: Physical Details, dtype: object, '"a developer\'s guide to data modeling for sql server : covering sql server 2005 and 2008"', '"eric johnson, joshua jones"', '"upper saddle river, nj : addison-wesley, [2008] ©2008"', '"books"', 9780321497642, '"addison-wesley microsoft technology series,"', '"nan"']
[]
[372    "microprocessor programming and applications for scientists and engineers"
Name: Title, dtype: object, 372    "smardzewski, richard r., 1942-"
Name: Author, dtype: object, '"amsterdam ; elsevier 

...

['"theory of quantum computation, communication, and cryptography : 7th conference, tqc 2012, tokyo, japan, may 17-19, 2012, revised selected papers"', 3394    "iwama, kazuo. | kawano, yasuhito. | murao, mio."
Name: Author, dtype: object, 3394    "berlin ; springer, â©2013."
Name: Publication, dtype: object, 3394    "electronic books. | conference papers and proceedings."
Name: Format, dtype: object, 3394    9783642356568
Name: ISBN, dtype: int64, 3394    "lecture notes in computer science ; 7582. lncs sublibrary. sl 1, theoretical computer science a...
Name: Series, dtype: object, 3394    "1 online resource."
Name: Physical Details, dtype: object, '"theory of quantum computation, communication, and cryptography : 7th conference, tqc 2012, tokyo, japan, may 17-19, 2012, revised selected papers"', '"kazuo iwama, yasuhito kawano, mio murao (eds.)"', '"berlin ; new york : springer, [2013] ©2013"', '"books"', 9783642356568, '"lecture notes in computer science ; 7582,lncs sublibrary. theore

['"financial cryptography and data security : fc 2017 international workshops, wahc, bitcoin, voting, wtsc, and ta, sliema, malta, april 7, 2017, revised selected papers"', '"edited by michael brenner, kurt rohloff, joseph bonneau, andrew miller, peter y.a. ryan, vanessa teague, andrea bracciali, massimiliano sala, federico pintore, markus jakobsson"', 3172    "cham, switzerland"
Name: Publication, dtype: object, 3172    "electronic books."
Name: Format, dtype: object, 3172    9783319702780
Name: ISBN, dtype: int64, 3172    "lecture notes in computer science ; 10323. 0302-9743 lncs sublibrary. sl 4, security and crypto...
Name: Series, dtype: object, 3172    "1 online resource (xxii, 636 pages)"
Name: Physical Details, dtype: object, '"financial cryptography and data security : fc 2017 international workshops, wahc, bitcoin, voting, wtsc, and ta, sliema, malta, april 7, 2017, revised selected papers"', '"edited by michael brenner, kurt rohloff, joseph bonneau, andrew miller, peter y.a.

[3117    "essentials of scientific computing : numerical methods in science and engineering"
Name: Title, dtype: object, 3117    "zalizniak, victor."
Name: Author, dtype: object, 3117    "chichester, uk"
Name: Publication, dtype: object, 3117    "electronic books."
Name: Format, dtype: object, 3117    9780857099365
Name: ISBN, dtype: int64, 3117    "nan"
Name: Series, dtype: object, 3117    "1 online resource (218 pages)"
Name: Physical Details, dtype: object, '"essentials of scientific computing : numerical methods in science and engineering"', '"victor zalizniak"', '"chichester, uk : horwood pub., 2008."', '"books"', 9781904275329, '"nan"', '"nan"']
[]
[]
[]
[2276    "homomorphic encryption and applications"
Name: Title, dtype: object, 2276    "yi, xun, (college teacher)"
Name: Author, dtype: object, 2276    "cham"
Name: Publication, dtype: object, 2276    "electronic books."
Name: Format, dtype: object, 2276    9783319122298
Name: ISBN, dtype: int64, 2276    "springerbriefs in compu

[4751    "biotechnology in japan : a comprehensive guide"
Name: Title, dtype: object, 4751    "schmid, rolf d."
Name: Author, dtype: object, 4751    "berlin, heidelberg"
Name: Publication, dtype: object, 4751    "electronic books."
Name: Format, dtype: object, 4751    9783642763397
Name: ISBN, dtype: int64, 4751    "nan"
Name: Series, dtype: object, 4751    "1 online resource (xiv, 769 pages)"
Name: Physical Details, dtype: object, '"biotechnology in japan : a comprehensive guide"', '"by rolf d. schmid"', '"berlin, heidelberg : springer berlin heidelberg, 1991."', '"books"', 9783642763397, '"nan"', '"nan"']
[]
[]
[]
[]
[]
[280    "nonlinear dynamical systems analysis for the behavioral sciences using real data"
Name: Title, dtype: object, 280    "guastello, stephen j. | gregson, r. a. m."
Name: Author, dtype: object, 280    "boca raton, fl"
Name: Publication, dtype: object, 280    "nan"
Name: Format, dtype: object, 280    9781439819975
Name: ISBN, dtype: int64, 280    "nan"
Name: Serie

['"data warehousing and knowledge discovery : 15th international conference, dawak 2013, prague, czech republic, august 26-29, 2013, proceedings"', 660    "bellatreche, ladjel, | mohania, mukesh,"
Name: Author, dtype: object, 660    "heidelberg"
Name: Publication, dtype: object, 660    "electronic books. | conference papers and proceedings."
Name: Format, dtype: object, 660    9783642401312
Name: ISBN, dtype: int64, 660    "lecture notes in computer science ; 8057. lncs sublibrary. sl 3, information systems and applic...
Name: Series, dtype: object, 660    "1 online resource (xiv, 374 pages)"
Name: Physical Details, dtype: object, '"data warehousing and knowledge discovery : 15th international conference, dawak 2013, prague, czech republic, august 26-29, 2013, proceedings"', '"ladjel bellatreche, mukesh k. mohania (eds.)"', '"heidelberg : springer, [2013]. ©2013"', '"books"', 9783642401312, '"lecture notes in computer science ; 8057,lncs sublibrary. information systems and applications

In [88]:
attr=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details']
df = pd.DataFrame(columns=attr)
for index, r in matched_set.iterrows():
    for  a in attr:
        left_name = 'ltable_' + a
        right_name = 'rtable_' + a
        if len(r[left_name]) < len(r[right_name]):
            df.iloc[index, attr.index(a)] = r[right_name]
        else:
            df.iloc[index, attr.index(a)] = r[left_name]

In [90]:
df = matched_set.iloc[:, 3:10]

In [91]:
df

,ltable_Title,ltable_Author,ltable_Publication,ltable_Format,ltable_ISBN,ltable_Series,ltable_Physical Details
0,"""statistical learning and data science""","""edited by mireille gettler summa ... [and others]""","""boca raton : crc press, [2012] ©2012""","""books""",9781439867631,"""series in computer science and data analysis,""","""nan"""
1,"""intelligent techniques for data science""","""rajendra akerkar, priti srinivas sajja""","""cham, switzerland : springer, 2016.""","""books""",9783319292069,"""nan""","""nan"""
2,"""algorithms for data science""","""brian steele, john chandler, swarna reddy""","""cham, switzerland : springer, 2016.""","""books""",9783319457970,"""nan""","""nan"""
3,"""data science at the command line""","""jeroen janssens""","""first edition. sebastopol, ca : o'reilly, 2014. ©2015""","""books""",9781491947852,"""nan""","""nan"""
5,"""introduction to hpc with mpi for data science""","""frank nielsen""","""cham : springer, 2016.""","""books""",9783319219035,"""undergraduate topics in computer science,""","""nan"""
6,"""data science using oracle data miner and oracle r enterprise : transform your business systems ...","""sibanjan das""","""berkeley, ca : apress, 2016. berkeley, ca : apress, 2016.""","""books""",9781484226148,"""nan""","""nan"""
7,"""the data science handbook""","""field cady""","""hoboken, nj : john wiley & sons, inc., 2017.""","""books""",9781119092933,"""nan""","""nan"""
8,"""data science : create teams that ask the right questions and deliver real value""","""doug rose""","""[berkeley, ca] : apress, 2016.""","""books""",9781484222539,"""nan""","""nan"""
9,"""spatial big data science classification techniques for earth observation imagery""","""zhe jiang, shashi shekhar""","""cham : springer, 2017.""","""books""",9783319601953,"""nan""","""nan"""
10,"""practical data science cookbook : 89 hands-on recipes to help you complete real-world data scie...","""tony ojeda, sean patrick murphy, benjamin bengfort, abhijit dasgupta""","""birmingham, uk : packt publishing ltd., september 2014. ©2014""","""books""",9781783980246,"""nan""","""nan"""


In [ ]:
df.rename(columns={'ltable_Title' : 'Title',
                  'ltable_Author' : 'Author',
                   ''
                  })

In [81]:
attr.index(a)

0

In [84]:
df.iloc[1,0]

IndexError: single positional indexer is out-of-bounds